# Prompt Declaration Language

Prompt engineering is difficult: minor variations in prompts have large impacts on the output of LLMs and prompts are model-dependent. In recent years <i> prompt programming languages </i> have emerged to bring discipline to prompt engineering. Many of them are embedded in an imperative language such as Python or TypeScript, making it difficult for users to directly interact with prompts and multi-turn LLM interactions.

The Prompt Declaration Language (PDL) is a YAML-based declarative approach to prompt programming, where prompts are at the forefront. PDL facilitates model chaining and tool use, abstracting away the plumbing necessary for such compositions, enables type checking of the input and output of models, and is based on LiteLLM to support a variety of model providers. PDL has been used with RAG, CoT, ReAct, and an agent for solving SWE-bench. PDL is [open-source](https://github.com/IBM/prompt-declaration-language) and works well with watsonx.ai and Granite models.

You can use PDL stand-alone or from a Python SDK or, as shown here, in a notebook via a notebook extension. In the cell output, model-generated text is rendered in green font, and tool-generated text is rendered in purple font.

In [ ]:
! pip install 'prompt-declaration-language[examples]'

In [1]:
%load_ext pdl.pdl_notebook_ext

## Model call

In PDL, the user specifies step-by-step the shape of data they want to generate. In the following, the `text` construct indicates a text block containing a prompt and a model call. Implicitly, PDL builds a background conversational context (list of role/content) which is used to make model calls. Each model call uses the context built so far as its input prompt.

In [ ]:
%%pdl --reset-context
description: Model call
text: 
- "What is the meaning of life?\n"
- model: replicate/ibm-granite/granite-3.2-8b-instruct
  parameters:
    stop: ["!"]
    include_stop_sequence: true

What is the meaning of life?
The meaning of life is a philosophical question that has been debated by many thinkers throughout history and is still a subject of discussion today. The answer to this question can vary greatly depending on one's personal beliefs, values, and experiences.


/Users/mvaziri/.pyenv/versions/3.12.5/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Model chaining
Model chaining can be done by simply adding to the list of models to call declaratively. Since this cell has the `%%pdl` cell magic without `--reset-context`, it executes in the context created by the previous cell.

In [3]:
%%pdl
text:
- "\nSay it like a poem\n"
- model: replicate/ibm-granite/granite-3.2-8b-instruct
- "\n\nWhat is the most important verse in this poem?\n"
- model: replicate/ibm-granite/granite-3.2-8b-instruct


Say it like a poem
The meaning of life,
A question that we all ask,
A mystery that still defies,
The answer that we all seek.
It's a question that's been asked,
Throughout history, by many,
And yet, it remains,
A mystery that we'll never know.
But, we can try to find,
The meaning of life,
And in doing so,
We can find our own way.


What is the most important verse in this poem?
The most important verse in this poem is "But, we can try to find, The meaning of life." This verse emphasizes the importance of trying to find the meaning of life, even though it may still be a mystery. It suggests that the search for meaning is an ongoing process, and that we can each find our own way to discover it.


## Chat templates

The following example shows a full-fledged chatbot. In PDL roles are high level annotations and PDL takes care of applying the appropriate chat templates. This example illustrates the use of control structures such as the repeat-until block and reading from files or stdin with the read block. The chatbot repeatedly prompts the user for a query, which it submits to a model, and stops when the query is quit.

In [4]:
%%pdl
text:
- role: system
  content: You are Granite, an AI language model developed by IBM in 2024. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior.
- "Type `quit` to exit this chatbot.\n"
- repeat:
    text:
    - read:
      message: ">>> "
      def: query
      contribute: [context]
    - model: replicate/ibm-granite/granite-3.2-8b-instruct
  until: ${ query == 'quit'}
  join:
    with: "\n\n"
role: user


You are Granite, an AI language model developed by IBM in 2024. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior.Type `quit` to exit this chatbot.


>>>  What is APR?


APR stands for Annual Percentage Rate, which is a measure of the total cost of borrowing money. It is the total amount of interest that a borrower pays over the life of a loan, expressed as a percentage of the loan amount. APR includes both the interest rate and any fees or charges associated with the loan, such as origination fees or prepayment penalties. It is a useful tool for comparing the cost of different loans and for understanding the true cost of borrowing money.




>>>  say it like a poem


APR stands for Annual Percentage Rate,
A measure of the total cost of borrowing,
It's the total amount of interest you pay,
Over the life of your loan, expressed in percentage.
APR includes fees and charges,
 origination fees, prepayment penalties, and more,
It's a useful tool for comparing loans,
And for understanding the true cost of borrowing.
But remember, not all loans are created equal,
APR doesn't tell the whole story,
It's important to read the fine print,
And understand all the terms and conditions.
So, before you sign on the dotted line,
Make sure you understand what you're getting,
APR is just one of many factors,
To consider when choosing a loan.




>>>  quit


Thank you for using my services. If you have any further questions or need additional assistance, please don't hesitate to ask.


## Chat templates

The first call to the model in the above program submits the following prompt. PDL takes care of applying the appropriate chat templates and tags, and builds the background context implicitly. Chat templates make your program easier to port across models, since you do not need to specify control tokens by hand. All the user has to do is list the models they want to chain, PDL takes care of the rest.

```
<|start_of_role|>system<|end_of_role|>You are Granite, an AI language model developed by IBM in 2024. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior.<|end_of_text|>
<|start_of_role|>user<|end_of_role|>Type `quit` to exit this chatbot.
What is APR?<|end_of_text|><|start_of_role|>assistant<|end_of_role|>
```

## Data pipeline

The following program shows a common prompting pattern: read some data, formulate a prompt using that data, submit to a model, and evaluate. In this program, we formulate a prompt for code explanation. The program first defines two variables: `code`, which holds the data we read, and `truth` for the ground truth. It then prints out the source code, formulates a prompts with the data, and calls a model to get an explanation. Finally, a Python code block uses the Levenshtein text distance metric and evaluate the explanation against the ground truth. This pipeline can similarly be applied to an entire data set to produce a jsonl file.

In [5]:
%%pdl --reset-context
description: Code explanation example
defs:
  CODE:
    read: ./data.yaml
    parser: yaml
  TRUTH:
    read: ./ground_truth.txt
text:
- "\n${ CODE.source_code }\n"
- model: replicate/ibm-granite/granite-3.2-8b-instruct
  def: EXPLANATION
  input: |
      Here is some info about the location of the function in the repo.
      repo: 
      ${ CODE.repo_info.repo }
      path: ${ CODE.repo_info.path }
      Function_name: ${ CODE.repo_info.function_name }


      Explain the following code:
      ```
      ${ CODE.source_code }```
- |


  EVALUATION:
  The similarity (Levenshtein) between this answer and the ground truth is:
- def: EVAL
  lang: python
  code: |
    import textdistance
    expl = """
    ${ EXPLANATION }
    """
    truth = """
    ${ TRUTH }
    """
    result = textdistance.levenshtein.normalized_similarity(expl, truth)


@SuppressWarnings("unchecked")
public static Map<String, String> deserializeOffsetMap(String lastSourceOffset) throws IOException {
  Map<String, String> offsetMap;
  if (lastSourceOffset == null || lastSourceOffset.isEmpty()) {    
    offsetMap = new HashMap<>();  
  } else {
    offsetMap = JSON_MAPPER.readValue(lastSourceOffset, Map.class);  
  }
  return offsetMap;
}

The code is a method in the `OffsetUtil` class that takes a string `lastSourceOffset` as input and returns a `Map` of `String` keys and `String` values. The method is annotated with `@SuppressWarnings("unchecked")` to suppress a warning about the unchecked conversion of the JSON string to a `Map` object.
 

 The method first checks if `lastSourceOffset` is `null` or empty. If it is, it creates a new empty `HashMap` and assigns it to the `offsetMap` variable. If `lastSourceOffset` is not `null` or empty, it uses the `JSON_MAPPER` object to deserialize the JSON string into a `Map` object. The `JSON_MAPPER` object is a

## Conclusion

Since prompts are at the forefront, PDL makes users more productive in their trial-and-error with LLMs. Try it!

https://github.com/IBM/prompt-declaration-language